<a href="https://colab.research.google.com/github/xtchen64/llm-playground/blob/main/notebook/xtchen64_PEFT_Finetune_Bloom560m_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using PEFT & bitsandbytes to finetune a LoRa checkpoint




This notebook is adapted from the youtube video: https://www.youtube.com/watch?v=Us5ZFp16PaU&t=559s.

Changes:
1. We are using a smaller bloom model (560m instead of 7b)
2. Adjusted the training configs to run for more epochs, which will hopefully get us a better model for the quote tagging task.

TODO:
- Use another dataset (e.g. All Obama tweets: https://github.com/fivethirtyeight/data/blob/master/twitter-ratio/BarackObama.csv)

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-428dbb65-7451-15a7-bbdf-150c7ba43c04)


### Setup the model

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-560m",
    load_in_8bit=True,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

### Freezing the original weights


In [5]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()  # enable computing gradients wrt inputs

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know which layers to target
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 786432 || all params: 560001024 || trainable%: 0.14043402892063284


## Data

In [8]:
import transformers
import random

from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")

In [9]:
type(data)

datasets.dataset_dict.DatasetDict

In [10]:
# split into train and validation
def split_dataset(dataset, train_ratio=0.7):
    """
    Splits the Hugging Face Dataset into training and validation sets.

    :param dataset: The Dataset instance to be split.
    :param train_ratio: The ratio of training data to total data (default is 0.7).
    :return: A dictionary with 'train' and 'validation' Dataset instances.
    """
    # Split the dataset into training and validation sets
    split_datasets = dataset.train_test_split(test_size=1 - train_ratio)

    return {"train": split_datasets["train"], "test": split_datasets["test"]}


split_data = split_dataset(data["train"], train_ratio=0.7)
train_data = split_data["train"]
test_data = split_data["test"]

In [11]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

train_data = train_data.map(merge_columns)
test_data = train_data.map(merge_columns)

test_data["prediction"][:5]

Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

["“Dance, when you're broken open. Dance, if you've torn the bandage off. Dance in the middle of the fighting. Dance in your blood. Dance when you're perfectly free.” ->: ['dance', 'inspirational']",
 "“People, generally, suck.” ->: ['humor']",
 "“Laughter is wine for the soul - laughter soft, or loud and deep, tinged through with seriousness - the hilarious declaration made by man that life is worth living.” ->: ['joy', 'laughter']",
 "“For every minute you are angry you lose sixty seconds of happiness.” ->: ['happiness']",
 "“Education: the path from cocky ignorance to miserable uncertainty.” ->: ['education']"]

In [12]:
train_data[0]

{'quote': "“Dance, when you're broken open. Dance, if you've torn the bandage off. Dance in the middle of the fighting. Dance in your blood. Dance when you're perfectly free.”",
 'author': 'Rumi',
 'tags': ['dance', 'inspirational'],
 'prediction': "“Dance, when you're broken open. Dance, if you've torn the bandage off. Dance in the middle of the fighting. Dance in your blood. Dance when you're perfectly free.” ->: ['dance', 'inspirational']"}

In [13]:
train_data = train_data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

In [14]:
test_data = test_data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/1755 [00:00<?, ? examples/s]

In [15]:
train_data

Dataset({
    features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
    num_rows: 1755
})

### Training

In [16]:
from transformers import Trainer, TrainingArguments


# Training arguments with an evaluation strategy
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    num_train_epochs=50,  # Set the number of training epochs
    learning_rate=1e-4,
    fp16=True,
    logging_steps=1,
    output_dir='outputs',
    evaluation_strategy="epoch",
)

# Initialize Trainer with training and evaluation datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,  # Add this line for evaluation dataset
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
0,3.842400,3.732070
2,2.808500,2.853227
4,2.401300,2.662641
6,2.596800,2.597874
8,2.564300,2.550403
10,2.598300,2.509858
12,2.414500,2.474958
14,2.463400,2.440245
16,2.214000,2.409871
18,2.620800,2.381990


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


KeyboardInterrupt: ignored

## Share adapters on the 🤗 Hub

In [17]:
model.push_to_hub("xtchen64/bloom-560m-lora-tagger-34-epoch",
                  use_auth_token=True,
                  commit_message="basic training",
                  private=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/xtchen64/bloom-560m-lora-tagger-34-epoch/commit/a50aa6cad5200e9b81006363c52eada02d7f7fa8', commit_message='basic training', commit_description='', oid='a50aa6cad5200e9b81006363c52eada02d7f7fa8', pr_url=None, pr_revision=None, pr_num=None)

## Load adapters from the Hub

In [19]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "xtchen64/bloom-560m-lora-tagger-34-epoch"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

## Inference

In [20]:
batch = tokenizer("“Sushi and Uni are both cute cats, but Sushi knows how to open the door so he's smarter. ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 “Sushi and Uni are both cute cats, but Sushi knows how to open the door so he's smarter. ->:  ['cats', 'humor', 'humor'], 'kats', 'cats', 'humor', 'humor'], 'kats', 'sushi', 'humor', 'humor'], 'kats'
